In [1]:
import pyspark.sql
from pyspark.sql import *
from pyspark.sql.functions import *

dir(pyspark.sql)

['Catalog',
 'Column',
 'DataFrame',
 'DataFrameNaFunctions',
 'DataFrameReader',
 'DataFrameStatFunctions',
 'DataFrameWriter',
 'DataFrameWriterV2',
 'GroupedData',
 'HiveContext',
 'Observation',
 'PandasCogroupedOps',
 'Row',
 'SQLContext',
 'SparkSession',
 'UDFRegistration',
 'Window',
 'WindowSpec',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'catalog',
 'column',
 'conf',
 'context',
 'dataframe',
 'functions',
 'group',
 'observation',
 'pandas',
 'readwriter',
 'session',
 'sql_formatter',
 'streaming',
 'types',
 'udf',
 'utils',
 'window']

In [3]:
spark = SparkSession.builder.getOrCreate() # on crée la session

In [6]:
data = spark.read.csv("demographics_de.csv", header=True, inferSchema=True) # on récupère le csv et on définit les paramètres

In [7]:
data.createOrReplaceTempView("demo")
request = spark.sql("select * from demo")
request.show()

+------------------+------+---------+----------+
|             state|gender|age_group|population|
+------------------+------+---------+----------+
|Baden-Wuerttemberg|female|    00-04|    261674|
|Baden-Wuerttemberg|female|    05-14|    490822|
|Baden-Wuerttemberg|female|    15-34|   1293488|
|Baden-Wuerttemberg|female|    35-59|   1919649|
|Baden-Wuerttemberg|female|    60-79|   1182736|
|Baden-Wuerttemberg|female|    80-99|    419471|
|Baden-Wuerttemberg|  male|    00-04|    274882|
|Baden-Wuerttemberg|  male|    05-14|    517387|
|Baden-Wuerttemberg|  male|    15-34|   1423367|
|Baden-Wuerttemberg|  male|    35-59|   1955828|
|Baden-Wuerttemberg|  male|    60-79|   1065669|
|Baden-Wuerttemberg|  male|    80-99|    264560|
|            Bayern|female|    00-04|    306378|
|            Bayern|female|    05-14|    560549|
|            Bayern|female|    15-34|   1507659|
|            Bayern|female|    35-59|   2310659|
|            Bayern|female|    60-79|   1421273|
|            Bayern|

In [8]:
table = spark.table("demo")
table.show()

+------------------+------+---------+----------+
|             state|gender|age_group|population|
+------------------+------+---------+----------+
|Baden-Wuerttemberg|female|    00-04|    261674|
|Baden-Wuerttemberg|female|    05-14|    490822|
|Baden-Wuerttemberg|female|    15-34|   1293488|
|Baden-Wuerttemberg|female|    35-59|   1919649|
|Baden-Wuerttemberg|female|    60-79|   1182736|
|Baden-Wuerttemberg|female|    80-99|    419471|
|Baden-Wuerttemberg|  male|    00-04|    274882|
|Baden-Wuerttemberg|  male|    05-14|    517387|
|Baden-Wuerttemberg|  male|    15-34|   1423367|
|Baden-Wuerttemberg|  male|    35-59|   1955828|
|Baden-Wuerttemberg|  male|    60-79|   1065669|
|Baden-Wuerttemberg|  male|    80-99|    264560|
|            Bayern|female|    00-04|    306378|
|            Bayern|female|    05-14|    560549|
|            Bayern|female|    15-34|   1507659|
|            Bayern|female|    35-59|   2310659|
|            Bayern|female|    60-79|   1421273|
|            Bayern|

## Exercice 1: Afficher les cinq premières lignes du DataFrame.



In [9]:
table.show(5)

+------------------+------+---------+----------+
|             state|gender|age_group|population|
+------------------+------+---------+----------+
|Baden-Wuerttemberg|female|    00-04|    261674|
|Baden-Wuerttemberg|female|    05-14|    490822|
|Baden-Wuerttemberg|female|    15-34|   1293488|
|Baden-Wuerttemberg|female|    35-59|   1919649|
|Baden-Wuerttemberg|female|    60-79|   1182736|
+------------------+------+---------+----------+
only showing top 5 rows



## Exercice 2: Compter le nombre total de lignes dans le DataFrame.



In [13]:
table.count()

192

## Exercice 3: Trouver le nombre distinct d'états (states) présents dans le DataFrame.



In [19]:
display_states = table.select("state").distinct().count()
print(display_states)

16


## Exercice 4: Calculer la somme de la population totale.



In [94]:
sum_population = table.select(col("population")).agg(sum(col("population"))).show()

+---------------+
|sum(population)|
+---------------+
|       83019213|
+---------------+



## Exercice 5: Afficher la population maximale par groupe d'âge (age_group).



In [37]:
max_poputaion = table.select("population", "age_group").groupby("age_group").agg(max(col("population"))).sort(col("age_group").desc()).show()

+---------+---------------+
|age_group|max(population)|
+---------+---------------+
|    80-99|         729148|
|    60-79|        2009976|
|    35-59|        3147565|
|    15-34|        2188068|
|    05-14|         832251|
|    00-04|         440708|
+---------+---------------+



## Exercice 6: Filtrer les lignes du DataFrame pour n'inclure que les états (states) dont la population dépasse 1 million.



In [54]:
states_above_1M = table.filter(col("population") > 1000000).sort(col('population').desc()).show()

+-------------------+------+---------+----------+
|              state|gender|age_group|population|
+-------------------+------+---------+----------+
|Nordrhein-Westfalen|female|    35-59|   3147565|
|Nordrhein-Westfalen|  male|    35-59|   3127984|
|             Bayern|  male|    35-59|   2348310|
|             Bayern|female|    35-59|   2310659|
|Nordrhein-Westfalen|  male|    15-34|   2188068|
|Nordrhein-Westfalen|female|    15-34|   2047533|
|Nordrhein-Westfalen|female|    60-79|   2009976|
| Baden-Wuerttemberg|  male|    35-59|   1955828|
| Baden-Wuerttemberg|female|    35-59|   1919649|
|Nordrhein-Westfalen|  male|    60-79|   1775275|
|             Bayern|  male|    15-34|   1639500|
|             Bayern|female|    15-34|   1507659|
| Baden-Wuerttemberg|  male|    15-34|   1423367|
|             Bayern|female|    60-79|   1421273|
|      Niedersachsen|female|    35-59|   1392647|
|      Niedersachsen|  male|    35-59|   1391992|
| Baden-Wuerttemberg|female|    15-34|   1293488|


## Exercice 7: Calculer la moyenne de la population pour chaque groupe d'âge (age_group).



In [73]:
mean_pop_by_age = table.select('population', 'age_group').groupby('age_group').agg(avg('population')).sort(col('age_group').desc()).show()

+---------+---------------+
|age_group|avg(population)|
+---------+---------------+
|    80-99|    168409.5625|
|    60-79|     562135.625|
|    35-59|   910557.46875|
|    15-34|   600409.78125|
|    05-14|    230138.0625|
|    00-04|   122699.90625|
+---------+---------------+



## Exercice 8: Trier le DataFrame par ordre décroissant de population.



In [78]:
sort_pop_desc = table.sort(col('population').desc()).show()

+-------------------+------+---------+----------+
|              state|gender|age_group|population|
+-------------------+------+---------+----------+
|Nordrhein-Westfalen|female|    35-59|   3147565|
|Nordrhein-Westfalen|  male|    35-59|   3127984|
|             Bayern|  male|    35-59|   2348310|
|             Bayern|female|    35-59|   2310659|
|Nordrhein-Westfalen|  male|    15-34|   2188068|
|Nordrhein-Westfalen|female|    15-34|   2047533|
|Nordrhein-Westfalen|female|    60-79|   2009976|
| Baden-Wuerttemberg|  male|    35-59|   1955828|
| Baden-Wuerttemberg|female|    35-59|   1919649|
|Nordrhein-Westfalen|  male|    60-79|   1775275|
|             Bayern|  male|    15-34|   1639500|
|             Bayern|female|    15-34|   1507659|
| Baden-Wuerttemberg|  male|    15-34|   1423367|
|             Bayern|female|    60-79|   1421273|
|      Niedersachsen|female|    35-59|   1392647|
|      Niedersachsen|  male|    35-59|   1391992|
| Baden-Wuerttemberg|female|    15-34|   1293488|


## Exercice 9: Sélectionner les lignes du DataFrame où l'âge (age_group) est "18-24" et le sexe (gender) est "F".



In [79]:
young_women = table.filter((col("age_group") == "15-34") & (col("gender") == "female")).show()

+--------------------+------+---------+----------+
|               state|gender|age_group|population|
+--------------------+------+---------+----------+
|  Baden-Wuerttemberg|female|    15-34|   1293488|
|              Bayern|female|    15-34|   1507659|
|              Berlin|female|    15-34|    477479|
|         Brandenburg|female|    15-34|    213065|
|              Bremen|female|    15-34|     83990|
|             Hamburg|female|    15-34|    246744|
|              Hessen|female|    15-34|    715254|
|Mecklenburg-Vorpo...|female|    15-34|    146826|
|       Niedersachsen|female|    15-34|    875509|
| Nordrhein-Westfalen|female|    15-34|   2047533|
|     Rheinland-Pfalz|female|    15-34|    448341|
|            Saarland|female|    15-34|    103235|
|             Sachsen|female|    15-34|    386154|
|      Sachsen-Anhalt|female|    15-34|    193271|
|  Schleswig-Holstein|female|    15-34|    305125|
|          Thueringen|female|    15-34|    191712|
+--------------------+------+--

## Exercice 10: Ajouter une nouvelle colonne "population_percentage" qui représente la proportion de la population par rapport à la population totale.



In [106]:
# add_column = table.withColumn("population_percentage", sum(table["population"] / sql_sum(table["population"]).over(Window.partitionBy()))
add_column = table.withColumn("population_percentage",col('population') * 100/sum_population).collect()[0][0]